In [ ]:
%load_ext blackcellmagic
%load_ext autoreload
%autoreload 2

#### Imports and setup

In [ ]:
import numpy as np
import pandas as pd
import h5py
import nibabel.freesurfer.mghformat as mgh
import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.formula.api as sm
from statsmodels.stats.multitest import multipletests

In [ ]:
from spacestream.core.constants import SUBJECTS, CORE_ROI_NAMES, ROI_COLORS
from spacestream.core.paths import DATA_PATH, RESULTS_PATH
from spacestream.utils.general_utils import sem
from spacestream.utils.get_utils import get_mapping

In [ ]:
seeds = [0,1,2,3,4]
hemis = ["lh","rh"]
model_types = [
    "MB_RN50_detection",
    "MB_RN50_action",
    "MB_RN50_categorization",
    "MB_RN18_detection",
    "MB_RN18_action",
    "MB_RN18_categorization",
    "TDANN_Supervised",
    "TDANN_SimCLR",
]

#### Load and format data

In [ ]:
def correct_for_voxel_noise_ceiling(NC, mapping):

    brain_r = np.sqrt(
        NC[mapping["winning_idx"].astype(int)] / 100
    )  # convert from R^2 to r
    mapping["winning_roi"] = mapping["winning_roi"].astype(np.float32)

    if np.sum(np.isinf(mapping["winning_test_corr"])) > 0:
        mapping["winning_test_corr"][np.isinf(info["winning_test_corr"])] = np.nan

    corrected = mapping["winning_test_corr"] / brain_r
    corrected[mapping["winning_test_corr"] == 0] = np.nan

    return corrected

In [ ]:
# read in data
long = {
    "model_type": [],
    "hemi": [],
    "subject": [],
    "ROIS": [],
    "result": [],
}


for hidx, hemi in enumerate(hemis):

    for sidx, subj in enumerate(SUBJECTS):

        # get ROI info
        mgh_file = mgh.load(DATA_PATH + "brains/" + hemi + ".ministreams.mgz")
        streams = mgh_file.get_fdata()[:, 0, 0].astype(int)
        # get noise ceiling estimates
        mgh_file = mgh.load(
            DATA_PATH + "brains/NC/subj" + subj + "/" + hemi + ".nc_3trials.mgh"
        )
        NC = mgh_file.get_fdata()[:, 0, 0]
        NC_trim = NC[streams != 0]
        NC_trim[NC_trim == 0] = np.nan  # Set all 0s to nans to avoid dividing by 0

        for mtype in model_types:

            if "TDANN" in mtype:
                supervised = 1 if "Supervised" in mtype else 0
                sw = "2.5" if "Supervised" in mtype else "0.25"
                temp_by_seed = np.zeros((len(seeds),3))
                for midx, seed in enumerate(seeds):
                    mapping = get_mapping(
                        subj_name="subj" + str(subj),
                        spatial_weight=sw,
                        model_seed=seed,
                        supervised=supervised,
                        hemi=hemi,
                    )
                    corrected = correct_for_voxel_noise_ceiling(NC_trim, mapping)
                    for ridx, r in enumerate(CORE_ROI_NAMES):
                        temp_by_seed[midx, ridx] = np.nanmean(
                            corrected[mapping["winning_roi"] == ridx + 5]
                        )

            else:  # not TDANNs
                mapping = get_mapping(
                    subj_name="subj" + str(subj),
                    model_seed=0,
                    hemi=hemi,
                    model_type="MB18" if "18" in mtype else "MB50",
                )

                corrected = correct_for_voxel_noise_ceiling(NC_trim, mapping)

            for ridx, r in enumerate(CORE_ROI_NAMES):
                long["model_type"].append(mtype)
                long["hemi"].append(hemi)
                long["subject"].append(subj)
                long["ROIS"].append(r)

                if "MB" not in mtype:
                    long["result"].append(
                        np.nanmean(
                            temp_by_seed[:,ridx],
                            axis=0,
                        )
                    )  # mean across seeds

                else:
                    if "categorization" in mtype:
                        t = 0
                    elif "action" in mtype:
                        t = 1
                    else:
                        t = 2

                    long["result"].append(
                        np.nanmean(
                            corrected[
                                (mapping["winning_roi"] == ridx + 5)
                                & (mapping["winning_task"] == t)
                            ]
                        )
                    )

In [ ]:
df = pd.DataFrame(long)
df = df.sort_values('ROIS') #just to get the plotting order right

In [ ]:
## load subject2subject estimates
s2s_corrected_by_stream= np.zeros((len(SUBJECTS),len(seeds),len(CORE_ROI_NAMES), len(hemis)))

for hidx, hemi in enumerate(hemis):
    
    for sidx, subj in enumerate(SUBJECTS):

        for seedix, seed in enumerate(seeds):

            load_path = (RESULTS_PATH
                            + "mappings/one_to_one/voxel2voxel/target_subj"
                            + subj
                            + "/mode_"
                            + hemi
                            + "_ministreams_HVA_only_radius5_max_iters100_constant_radius_2.0dist_cutoff_constant_dist_cutoff_spherical"
                            + ("_CV_seed" + str(seed))
                            + "_final_voxel2voxel_correlation_info.hdf5"
                        )
            with h5py.File(load_path, "r") as f:

                for r, ridx in enumerate(CORE_ROI_NAMES):
                    s2s_corrected_by_stream[sidx,seedix,r,hidx] =  np.nanmean(f['corrected_test_corr'][:][f['winning_roi'][:] == (2-r)+5])
across_seed_corrected_mean = np.mean(np.mean(s2s_corrected_by_stream,axis=-1),axis=1)

In [ ]:
across_seed_corrected_mean

#### Plotting

In [ ]:
sns.set_theme(style="ticks")
for gidx, group in enumerate(model_types):

    group_df = df[df["model_type"].isin([group])]
    fig, ax = plt.subplots(figsize=(1, 5))
    sns.barplot(
        x="model_type",
        y="result",
        hue="ROIS",
        palette=ROI_COLORS,
        data=group_df,
        ax=ax,
        errorbar="se",
        errwidth=2,
        linewidth=3,
    )
    sns.stripplot(
        x="model_type",
        y="result",
        hue="ROIS",
        palette=ROI_COLORS,
        data=group_df[group_df["hemi"].isin(["lh"])],
        marker="^",
        ax=ax,
        size=5,
        dodge=True,
    )
    sns.stripplot(
        x="model_type",
        y="result",
        hue="ROIS",
        palette=ROI_COLORS,
        data=group_df[group_df["hemi"].isin(["rh"])],
        ax=ax,
        size=5,
        dodge=True,
    )

    for points in ax.collections:

        points.set_edgecolor("white")
        points.set_linewidth(0.75)

    # subj2subj NC bars
    xmin = 0.1
    xmax = 0.28
    for ridx, ROI in enumerate(CORE_ROI_NAMES):
        vals = across_seed_corrected_mean[:, ridx]
        y1 = np.mean(vals) - sem(vals)
        y2 = np.mean(vals) + sem(vals)
        ax.axhspan(
            y1,
            y2,
            xmin=xmin + (ridx * 0.26),
            xmax=xmax + (ridx * 0.26),
            alpha=0.9,
            color="lightgray",  # oi_palette[ridx], alpha=0.2
        )

    ax.legend([], [], frameon=False)

    ax.set_xlabel("", fontsize=20)
    ax.set_ylabel("Functional similarity (r)", fontsize=15)
    ax.tick_params(labelsize=15)
    ax.axhline(33, c="k", linestyle="--")

    ax.spines["right"].set_visible(False)
    ax.spines["top"].set_visible(False)
    plt.ylim(bottom = 0.0, top=0.4)
    
    #save_path = RESULTS_PATH + "figures/2c_" + group + ".svg"
    #
    # fig.savefig(save_path, dpi=100)

In [ ]:
d_sub = across_seed_corrected_mean[:,0]
l_sub = across_seed_corrected_mean[:,1]
v_sub = across_seed_corrected_mean[:,2]

#### Statistics

In [ ]:
# object categorization is the best "multi-behavior model" for all three streams so we will use only those values for stats
df = df[
    df["model_type"].isin(
        [
            "MB_RN18_categorization",
            "MB_RN50_categorization",
            "TDANN_Supervised",
            "TDANN_SimCLR",
        ]
    )
]
df = df.replace("MB_RN18_categorization", value="MB18")
df = df.replace("MB_RN50_categorization", value="MB50")

# set up separate models by ROI
dorsal_df = df[df["ROIS"] == "Dorsal"]
lateral_df = df[df["ROIS"] == "Lateral"]
ventral_df = df[df["ROIS"] == "Ventral"]

In [ ]:
d_m = dorsal_df[dorsal_df["model_type"]=="MB18"].groupby('subject')['result'].mean().reset_index()['result']
l_m = lateral_df[lateral_df["model_type"]=="MB18"].groupby('subject')['result'].mean().reset_index()['result']
v_m = ventral_df[ventral_df["model_type"]=="MB18"].groupby('subject')['result'].mean().reset_index()['result']

In [ ]:
from scipy.stats import ttest_rel

In [ ]:
print(ttest_rel(d_m,d_sub)[1]*18)
print(ttest_rel(l_m,l_sub)[1]*18)
print(ttest_rel(v_m,v_sub)[1]*18)

In [ ]:
# Dorsal
dorsal_mod = sm.mixedlm('result~model_type', data = dorsal_df, groups=dorsal_df["subject"]).fit()
print(dorsal_mod.summary())

res = pd.concat([dorsal_mod.params,dorsal_mod.pvalues],axis=1)
res.columns=['coefficient','pvalues']
print(res)
res = res[res.index.str.contains('model_type')]
res['corrected_p'] = multipletests(res['pvalues'],method="bonferroni")[1]
print(res)

In [ ]:
# Lateral
lateral_mod = sm.mixedlm('result~model_type', data = lateral_df, groups=lateral_df["subject"]).fit()
print(lateral_mod.summary())

res = pd.concat([lateral_mod.params,lateral_mod.pvalues],axis=1)
res.columns=['coefficient','pvalues']
print(res)

res = res[res.index.str.contains('model_type')]

res['corrected_p'] = multipletests(res['pvalues'],method="bonferroni")[1]
print(res)

In [ ]:
# Ventral
ventral_mod = sm.mixedlm('result~model_type', data = ventral_df, groups=ventral_df["subject"]).fit()
print(ventral_mod.summary())

res = pd.concat([ventral_mod.params,ventral_mod.pvalues],axis=1)
res.columns=['coefficient','pvalues']
print(res)

res = res[res.index.str.contains('model_type')]

res['corrected_p'] = multipletests(res['pvalues'],method="bonferroni")[1]
print(res)